In [1]:
import pandas as pd 
import numpy as np
from datetime import datetime

DATASET_PATH = './../dataset/'
PREPROCESSED_PATH = './../preprocessed/'

#Method for merging data
def merge_data(df):
    features = pd.read_csv(DATASET_PATH + 'features.csv')
    stores_data = pd.read_csv(DATASET_PATH + 'stores.csv')
    df = pd.merge(df, features, on=['Store','Date','IsHoliday'],
                  how='inner')
    df = pd.merge(df, stores_data, on=['Store'],
                  how='inner')
    return df

# Calc difference between two dates and return an integer
def diff(start, end):
    x = pd.to_datetime(end) - pd.to_datetime(start)
    return int(x / np.timedelta64(1, '7D'))

In [ ]:
# Reading Data
train = merge_data(pd.read_csv(DATASET_PATH + 'train.csv'))

# Generating Date type from string Date
train['DateType'] = [datetime.strptime(date, '%Y-%m-%d').date() for date in train['Date'].astype(str).values.tolist()]

groupped = train.groupby(by=['Store', 'Dept'])

index = pd.date_range(train.DateType.min(), periods=diff(train.DateType.min(), train.DateType.max()), freq='7D')
timeSeries = pd.DataFrame(index=index, columns=[])

for name, group in groupped:
    aux = pd.DataFrame(group[['Weekly_Sales', 'DateType']].copy())
    aux.columns = [name, 'DateType']
    aux.set_index('DateType', inplace=True)
    aux = aux.fillna(0)

    timeSeries = timeSeries.join(aux, how='outer')

timeSeries.to_csv(PREPROCESSED_PATH + 'timeseries.csv')